In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

sales_train = pd.read_csv('./sales_train.csv')
shops = pd.read_csv('./shops.csv')
items = pd.read_csv('./items.csv')
item_categories = pd.read_csv('./item_categories.csv')
test = pd.read_csv('./test.csv')
submission = pd.read_csv('./sample_submission.csv')

In [2]:
sales_train = sales_train.rename(columns={'date':'날짜',
                                         'date_block_num' : '월id',
                                         'shop_id':'상점id',
                                         'item_id' : '상품id',
                                         'item_price':'판매가',
                                         'item_cnt_day':'판매량'})

shops = shops.rename(columns={'shop_name':'상점명', 'shop_id':'상점id'})

items = items.rename(columns={'item_name':'상품명',
                             'item_id':'상품id',
                             'item_category_id':'상품분류id'})

item_categories = item_categories.rename(columns={'item_category_name':'상품분류명',
                                                  'item_category_id':'상품분류id'})

test = test.rename(columns={'shop_id':'상점id', 'item_id':'상품id'})

In [3]:
def downcast(df, verbose=True):
    start_men = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_men = df.memory_usage().sum() / 1024**2
    if verbose:
        print('{:.1f}% 압축됨'.format(100*(start_men - end_men)/start_men))
        
    return df

In [4]:
all_df = [sales_train, shops, items, item_categories, test]
for df in all_df:
    df = downcast(df)

62.5% 압축됨
38.6% 압축됨
54.2% 압축됨
39.9% 압축됨
70.8% 압축됨


#### 이상치 제거
- 판매가, 판매량 음수 제거
- 판매가 50000 이상, 판매량 1000 이상 데이터 제거

In [5]:
sales_train = sales_train[sales_train['판매가']>0]
sales_train = sales_train[sales_train['판매가']<50000]

sales_train = sales_train[sales_train['판매량']>0]
sales_train = sales_train[sales_train['판매량']<1000]

In [6]:
sales_train.loc[sales_train['상점id']==0, '상점id']=57
sales_train.loc[sales_train['상점id']==1, '상점id']=58
sales_train.loc[sales_train['상점id']==10, '상점id']=11
sales_train.loc[sales_train['상점id']==39, '상점id']=40

test.loc[test['상점id']==0, '상점id']=57
test.loc[test['상점id']==1, '상점id']=58
test.loc[test['상점id']==10, '상점id']=11
test.loc[test['상점id']==39, '상점id']=40

- 상점명의 첫 단어가 도시

In [7]:
shops['도시'] = shops['상점명'].apply(lambda x:x.split()[0])

In [8]:
shops['도시'].unique()

array(['!Якутск', 'Адыгея', 'Балашиха', 'Волжский', 'Вологда', 'Воронеж',
       'Выездная', 'Жуковский', 'Интернет-магазин', 'Казань', 'Калуга',
       'Коломна', 'Красноярск', 'Курск', 'Москва', 'Мытищи', 'Н.Новгород',
       'Новосибирск', 'Омск', 'РостовНаДону', 'СПб', 'Самара', 'Сергиев',
       'Сургут', 'Томск', 'Тюмень', 'Уфа', 'Химки', 'Цифровой', 'Чехов',
       'Якутск', 'Ярославль'], dtype=object)

In [9]:
shops.loc[shops['도시']=='!Якутск', '도시'] = 'Якутск'

In [10]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
shops['도시'] = label_encoder.fit_transform(shops['도시'])

In [11]:
# 상점id에 상점명 내포되어 있으므로 상점명 제거
shops = shops.drop('상점명', axis=1)

shops.head()

,상점id,도시
0,0,29
1,1,29
2,2,0
3,3,1
4,4,2


In [12]:
items = items.drop('상품명', axis=1)

- '첫 판매월' 변수 만들기

In [13]:
items['첫 판매월'] = sales_train.groupby('상품id').agg({'월id':'min'})['월id']

items.head()

,상품id,상품분류id,첫 판매월
0,0,40,20.0
1,1,76,15.0
2,2,40,19.0
3,3,40,18.0
4,4,40,20.0


In [14]:
items[items['첫 판매월'].isna()]

,상품id,상품분류id,첫 판매월
83,83,40,NaN
140,140,45,NaN
168,168,44,NaN
173,173,45,NaN
204,204,44,NaN
...,...,...,...
21974,21974,61,NaN
21975,21975,61,NaN
22022,22022,40,NaN
22035,22035,40,NaN


- 해당 상품들은 한 번도 판매된 적이 없음
- 처음 팔린 달을 2015년 11월 가정 -> 결측값을 34로 대체

In [15]:
items['첫 판매월'] = items['첫 판매월'].fillna(34)

#### 대분류 파생 변수 만들기
### 범주형 데이터가 지나치게 세밀할 경우 제대로 훈련되지 못할 수 있다!

In [16]:
item_categories['대분류'] = item_categories['상품분류명'].apply(lambda x:x.split()[0])

In [17]:
item_categories['대분류'].value_counts()

Игры          14
Книги         13
Подарки       12
Игровые        8
Аксессуары     7
Музыка         6
Программы      6
Карты          5
Кино           5
Служебные      2
Чистые         2
PC             1
Билеты         1
Доставка       1
Элементы       1
Name: 대분류, dtype: int64

- 'Игры' : 게임
- 'Книги' : 책
- 'Подарки' : 선물

- 고윳값이 5개 미만인 대분류는 모두 'etc'로 변경

In [18]:
def make_etc(x):
    if len(item_categories[item_categories['대분류']==x]) >= 5:
        return x
    else:
        return 'etc'
    
item_categories['대분류'] = item_categories['대분류'].apply(make_etc)

In [19]:
item_categories.head()

,상품분류명,상품분류id,대분류
0,PC - Гарнитуры/Наушники,0,etc
1,Аксессуары - PS2,1,Аксессуары
2,Аксессуары - PS3,2,Аксессуары
3,Аксессуары - PS4,3,Аксессуары
4,Аксессуары - PSP,4,Аксессуары


In [20]:
item_categories['대분류'] = label_encoder.fit_transform(item_categories['대분류'])
item_categories = item_categories.drop('상품분류명', axis=1)

#### 데이터 조합

In [21]:
from itertools import product

train = []
# 월id, 상점id, 상품id 조합 생성
for i in sales_train['월id'].unique():
    all_shop = sales_train.loc[sales_train['월id']==i, '상점id'].unique()
    all_item = sales_train.loc[sales_train['월id']==i, '상품id'].unique()
    train.append(np.array(list(product([i], all_shop, all_item))))
    
idx_features = ['월id', '상점id', '상품id'] # 기준피처
train = pd.DataFrame(np.vstack(train), columns=idx_features)

train.head()

,월id,상점id,상품id
0,0,59,22154
1,0,59,2552
2,0,59,2554
3,0,59,2555
4,0,59,2564


#### 월간 판매량, 평균 판매량 만들기

In [22]:
group = sales_train.groupby(idx_features).agg({'판매량':'sum',
                                             '판매가':'mean'})

group = group.reset_index()
group = group.rename(columns={'판매량':'월간 판매량', '판매가':'평균 판매가'})

train = train.merge(group, on=idx_features, how='left')

train.head()

,월id,상점id,상품id,월간 판매량,평균 판매가
0,0,59,22154,1.0,999.0
1,0,59,2552,NaN,NaN
2,0,59,2554,NaN,NaN
3,0,59,2555,NaN,NaN
4,0,59,2564,NaN,NaN


In [23]:
import gc

del group
gc.collect()

0

#### 기준 변수별 상품 판매건수 : 당일에 판매한 개수

In [24]:
group = sales_train.groupby(idx_features).agg({'판매량':'count'})
group = group.reset_index()
group = group.rename(columns={'판매량':'판매건수'})

train = train.merge(group, on=idx_features, how='left')

del group, sales_train
gc.collect()

train.head()

,월id,상점id,상품id,월간 판매량,평균 판매가,판매건수
0,0,59,22154,1.0,999.0,1.0
1,0,59,2552,NaN,NaN,NaN
2,0,59,2554,NaN,NaN,NaN
3,0,59,2555,NaN,NaN,NaN
4,0,59,2564,NaN,NaN,NaN


In [25]:
test['월id'] = 34

all_data = pd.concat([train, test.drop('ID', axis=1)],
                    ignore_index=True, keys=idx_features)

all_data = all_data.fillna(0)

all_data.head()

,월id,상점id,상품id,월간 판매량,평균 판매가,판매건수
0,0,59,22154,1.0,999.0,1.0
1,0,59,2552,0.0,0.0,0.0
2,0,59,2554,0.0,0.0,0.0
3,0,59,2555,0.0,0.0,0.0
4,0,59,2564,0.0,0.0,0.0


In [26]:
all_data = all_data.merge(shops, on='상점id', how='left')
all_data = all_data.merge(items, on='상품id', how='left')
all_data = all_data.merge(item_categories, on='상품분류id', how='left')

all_data = downcast(all_data)

del shops, items, item_categories
gc.collect();

59.6% 압축됨


#### 시차 변수 만들기
- 시계열 문제에서 자주 만들며 성능 개선에 도움을 줌

#### 월간 평균 판매량 변수
- 상점별 월간 평균 판매량
- 상점의 상품별 월간 평균 판매량

In [27]:
def add_mean_features(df, mean_features, idx_features):
    # 기준 변수
    assert (idx_features[0] == '월id') and len(idx_features) in [2,3]
    
    # 파생 변수명 설정
    if len(idx_features) == 2:
        feature_name = idx_features[1] + '별 평균 판매량'
    else:
        feature_name = idx_features[1] + ' ' + idx_features[2] + '별 평균 판매량'
        
    group = df.groupby(idx_features).agg({'월간 판매량':'mean'})
    group = group.reset_index()
    group = group.rename(columns={'월간 판매량':feature_name})
    
    df = df.merge(group, on=idx_features, how='left')
    df = downcast(df, verbose=False) # 압축 문구 출력되지 않음
    
    mean_features.append(feature_name)
    
    del group
    gc.collect()
    
    return df, mean_features

In [28]:
item_mean_features=[]

all_data, item_mean_features = add_mean_features(df=all_data,
                                                mean_features=item_mean_features,
                                                idx_features=['월id', '상품id'])

all_data, item_mean_features = add_mean_features(df=all_data,
                                                mean_features=item_mean_features,
                                                idx_features=['월id', '상품id', '도시'])

In [29]:
item_mean_features

['상품id별 평균 판매량', '상품id 도시별 평균 판매량']

In [30]:
shop_mean_features = []

all_data, shop_mean_features = add_mean_features(df=all_data,
                                                mean_features=shop_mean_features,
                                                idx_features=['월id', '상점id','상품분류id'])

In [31]:
shop_mean_features

['상점id 상품분류id별 평균 판매량']

In [32]:
def add_lag_features(df, lag_features_to_clip, idx_features, 
                     lag_feature, nlags=3, clip=False):
    # 시차 피처 생성에 필요한 DataFrame 부분만 복사 
    df_temp = df[idx_features + [lag_feature]].copy() 

    # 시차 피처 생성 
    for i in range(1, nlags+1):
        # 시차 피처명 
        lag_feature_name = lag_feature +'_시차' + str(i)
        # df_temp 열 이름 설정 
        df_temp.columns = idx_features + [lag_feature_name]
        # df_temp의 date_block_num 피처에 1 더하기 
        df_temp['월id'] += 1
        # idx_feature를 기준으로 df와 df_temp 병합하기 
        df = df.merge(df_temp.drop_duplicates(), 
                      on=idx_features, 
                      how='left')
        # 결측값 0으로 대체 
        df[lag_feature_name] = df[lag_feature_name].fillna(0)
        # 0 ~ 20 사이로 제한할 시차 피처명을 lag_features_to_clip에 추가 
        if clip: 
            lag_features_to_clip.append(lag_feature_name)

    # 데이터 다운캐스팅
    df = downcast(df, False)
    # 가비지 컬렉션
    del df_temp
    gc.collect()

    return df, lag_features_to_clip

In [33]:
lag_features_to_clip = []
idx_features = ['월id', '상점id', '상품id']

all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                 lag_features_to_clip=lag_features_to_clip,
                                                 idx_features=idx_features,
                                                 lag_feature='월간 판매량',
                                                 nlags=3, clip=True)

all_data.head().T

,0,1,2,3,4
월id,0.000000,0.000000,0.000000,0.000000,0.000000
상점id,59.000000,59.000000,59.000000,59.000000,59.000000
상품id,22154.000000,2552.000000,2554.000000,2555.000000,2564.000000
월간 판매량,1.000000,0.000000,0.000000,0.000000,0.000000
평균 판매가,999.000000,0.000000,0.000000,0.000000,0.000000
판매건수,1.000000,0.000000,0.000000,0.000000,0.000000
도시,30.000000,30.000000,30.000000,30.000000,30.000000
상품분류id,37.000000,58.000000,58.000000,56.000000,59.000000
첫 판매월,0.000000,0.000000,0.000000,0.000000,0.000000
대분류,5.000000,7.000000,7.000000,7.000000,7.000000


In [34]:
lag_features_to_clip

['월간 판매량_시차1', '월간 판매량_시차2', '월간 판매량_시차3']

In [35]:
all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                 lag_features_to_clip=lag_features_to_clip,
                                                 idx_features=idx_features,
                                                 lag_feature='판매건수', nlags=3)

all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                 lag_features_to_clip=lag_features_to_clip,
                                                 idx_features=idx_features,
                                                 lag_feature='평균 판매가', nlags=3)

In [36]:
for item_mean_feature in item_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                     lag_features_to_clip=lag_features_to_clip,
                                                     idx_features=idx_features,
                                                     lag_feature = item_mean_feature,
                                                     nlags=3, clip=True)
all_data = all_data.drop(item_mean_features, axis=1)

In [37]:
for shop_mean_feature in shop_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data,
                                                     lag_features_to_clip=lag_features_to_clip,
                                                     idx_features=idx_features,
                                                     lag_feature = shop_mean_feature,
                                                     nlags=3, clip=True)
all_data = all_data.drop(shop_mean_features, axis=1)

In [38]:
all_data.head().T

,0,1,2,3,4
월id,0.0,0.0,0.0,0.0,0.0
상점id,59.0,59.0,59.0,59.0,59.0
상품id,22154.0,2552.0,2554.0,2555.0,2564.0
월간 판매량,1.0,0.0,0.0,0.0,0.0
평균 판매가,999.0,0.0,0.0,0.0,0.0
판매건수,1.0,0.0,0.0,0.0,0.0
도시,30.0,30.0,30.0,30.0,30.0
상품분류id,37.0,58.0,58.0,56.0,59.0
첫 판매월,0.0,0.0,0.0,0.0,0.0
대분류,5.0,7.0,7.0,7.0,7.0


In [39]:
# 결측값 제거
all_data = all_data.drop(all_data[all_data['월id']<3].index)

In [40]:
all_data['월간 판매량 시차평균'] = all_data[['월간 판매량_시차1',
                                   '월간 판매량_시차2', '월간 판매량_시차3']].mean(axis=1)

In [42]:
all_data[lag_features_to_clip + ['월간 판매량', '월간 판매량 시차평균']] = \
all_data[lag_features_to_clip + ['월간 판매량', '월간 판매량 시차평균']].clip(0,20)

#### 시차 변화량
- 시차변화량1 = 월간 판매량_시차1 / 월간 판매량_시차2
- 시차변화량2 = 월간 판매량_시차2 / 월간 판매량_시차3

In [43]:
all_data['시차변화량1'] = all_data['월간 판매량_시차1'] / all_data['월간 판매량_시차2']
# 0으로 나눈 값이 무한대인 경우를 0으로 바꿈
all_data['시차변화량1'] = all_data['시차변화량1'].replace([np.inf, -np.inf], np.nan).fillna(0)

all_data['시차변화량2'] = all_data['월간 판매량_시차2'] / all_data['월간 판매량_시차3']
all_data['시차변화량2'] = all_data['시차변화량2'].replace([np.inf, -np.inf], np.nan).fillna(0)

In [44]:
# 신상품인지 여부 
all_data['신상여부'] = all_data['첫 판매월'] == all_data['월id']

# 첫 판매 후 경과 시간
all_data['첫 판매 후 기간'] = all_data['월id'] - all_data['첫 판매월']

# 월
all_data['월'] = all_data['월id']%12

In [45]:
all_data = all_data.drop(['첫 판매월', '평균 판매가', '판매건수'], axis=1)

all_data = downcast(all_data, False)

In [46]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9904582 entries, 1122386 to 11026967
Data columns (total 31 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   월id                      int8   
 1   상점id                     int8   
 2   상품id                     int16  
 3   월간 판매량                   int8   
 4   도시                       int8   
 5   상품분류id                   int8   
 6   대분류                      int8   
 7   월간 판매량_시차1               int8   
 8   월간 판매량_시차2               int8   
 9   월간 판매량_시차3               int8   
 10  판매건수_시차1                 int8   
 11  판매건수_시차2                 int8   
 12  판매건수_시차3                 int8   
 13  평균 판매가_시차1               float32
 14  평균 판매가_시차2               float32
 15  평균 판매가_시차3               float32
 16  상품id별 평균 판매량_시차1         float32
 17  상품id별 평균 판매량_시차2         float32
 18  상품id별 평균 판매량_시차3         float32
 19  상품id 도시별 평균 판매량_시차1      float32
 20  상품id 도시별 평균 판매량_시차2      float32
 21  상

In [47]:
X_train = all_data[all_data['월id']<33]
X_train = X_train.drop(['월간 판매량'], axis=1)

X_valid = all_data[all_data['월id']==33]
X_valid = X_valid.drop(['월간 판매량'], axis=1)

X_test = all_data[all_data['월id']== 34]
X_test = X_test.drop(['월간 판매량'], axis=1)

y_train = all_data[all_data['월id']<33]['월간 판매량']
y_train = y_train.clip(0,20) # 0~20으로 제한

y_valid = all_data[all_data['월id']==33]['월간 판매량']
y_valid = y_valid.clip(0,20)

del all_data
gc.collect();

In [48]:
import lightgbm as lgb

params = {'metric':'rmse',
         'num_leaves':225,
         'learning_rate':0.005,
          'feature_fraction':0.75,
          'bagging_fraction':0.75,
          'bagging_freq':5,
         'force_col_wise':True,
         'random_state':10}

# 범주형 피처
cat_features = ['상점id', '도시','상품분류id','대분류','월']

dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)

lgb_model = lgb.train(params=params, train_set = dtrain,
                     num_boost_round=1500, valid_sets=(dtrain, dvalid),
                      early_stopping_rounds=150, categorical_feature=cat_features, 
                      verbose_eval=100)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 3902
[LightGBM] [Info] Number of data points in the train set: 9452298, number of used features: 30
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 0.297707
Training until validation scores don't improve for 150 rounds
[100]	training's rmse: 1.01466	valid_1's rmse: 0.993052
[200]	training's rmse: 0.914476	valid_1's rmse: 0.930158
[300]	training's rmse: 0.864098	valid_1's rmse: 0.905952
[400]	training's rmse: 0.836335	valid_1's rmse: 0.895957
[500]	training's rmse: 0.818137	valid_1's rmse: 0.892074
[600]	training's rmse: 0.805371	valid_1's rmse: 0.890762
[700]	training's rmse: 0.796058	valid_1's rmse: 0.889782
[800]	training's rmse: 0.788665	valid_1's rmse: 0.889377
[900]	training's rmse: 0.782568	valid_1's rmse: 0.889316
Early stopping, best iteration is:
[754]	training's rmse: 0.791945	valid_1's rmse: 0

In [49]:
preds = lgb_model.predict(X_test).clip(0,20)

submission['item_cnt_month'] = preds
submission.to_csv('submission2.csv', index=False)

In [50]:
del X_train, y_train, X_valid, y_valid, X_test, lgb_model, dtrain, dvalid
gc.collect()

4